In [57]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [58]:
import sys
sys.path.append('../src/')
sys.path.append('../../Brain_DNA/')

import os
import numpy as np
import pandas as pd
import torch
import utils as utils
import pickle
from impf_models.MLP.Model import Impf_DNAMLP
from torch.nn.functional import softmax

In [59]:
alg = 'RF'
group = 'Sella'
test_fold = '3.0'
fold_model = '3.4'
device = 'cuda' if torch.cuda.is_available() else 'cpu'

TEST_CSV_FILE = os.path.join('./test_data/', group, f'{test_fold}_test.csv')
df_test = pd.read_csv(TEST_CSV_FILE, index_col=0).fillna(0)
df_test.tail(5)

,cg22054918,cg16476975,cg25570913,cg03940848,cg02809746,cg01517680,cg04110886,cg18222083,cg24545961,cg05649108,...,cg18817900,cg04575155,cg13948824,cg17616554,cg27572510,cg22513691,cg00743629,cg13621317,label,class_group
3999078088_R05C01,0.074333,0.147507,0.010183,0.163439,0.032703,0.369905,1.000000,0.038912,0.367218,0.894081,...,0.859707,0.751199,0.888488,0.124954,0.899125,0.642912,0.961547,0.948671,"PITAD, FSH LH",Sella
3998523003_R04C01,0.345325,0.127623,0.586592,0.161607,0.278602,0.335895,0.346934,0.450223,0.460586,0.407217,...,0.750053,0.822987,0.601228,0.173946,0.920081,0.885622,0.466970,0.741451,"CONTR, HYPTHAL",Control
3998523003_R05C01,0.258860,0.173861,0.447441,0.271453,0.216092,0.430823,0.358002,0.444759,0.526878,0.516550,...,0.909444,0.765896,0.696270,0.121455,0.884375,0.883335,0.455505,0.830911,"CONTR, HYPTHAL",Control
3998523022_R03C01,0.056991,0.812723,0.163284,0.047708,0.034947,0.496456,0.962376,0.016874,0.929404,0.853016,...,0.611719,0.575597,0.935100,0.221665,0.194714,0.655032,0.109734,0.857919,"PITAD, TSH",Sella
3998523022_R04C01,0.087497,0.092697,0.064110,0.029223,0.243997,0.161502,1.000000,0.074434,0.660367,0.063579,...,0.422638,0.367502,0.920804,0.182162,0.237444,0.726276,0.171458,0.862736,"PITAD, TSH",Sella


In [60]:
IMPF_PATH = os.path.join('../data/processed/important_features/', alg, group, f"{fold_model.split('.')[0]}.0_combined.pkl")
print(IMPF_PATH)
with open(IMPF_PATH, 'rb') as file:
    impf = pickle.load(file)  
print(len(impf))
print(list(impf)[:5])
print(sorted(list(impf))[:5])
pd.DataFrame(list(impf)).T

../data/processed/important_features/RF\Sella\3.0_combined.pkl
2049
['cg23685650', 'cg10122865', 'cg05404698', 'cg10416206', 'cg23044053']
['cg00017157', 'cg00018128', 'cg00031256', 'cg00033213', 'cg00054971']


,0,1,2,3,4,5,6,7,8,9,...,2039,2040,2041,2042,2043,2044,2045,2046,2047,2048
0,cg23685650,cg10122865,cg05404698,cg10416206,cg23044053,cg13492692,cg08529345,cg27319917,cg07167423,cg13278738,...,cg02395846,cg23051392,cg03763873,cg04569429,cg07333715,cg26196480,cg10224600,cg11206381,cg01434649,cg11379081


In [42]:
test_features = np.array(df_test.loc[:, list(impf)])
test_labels = np.array(df_test.iloc[:,-1])

In [43]:
rand = np.random.randint(test_features.shape[0])
sample_feature = test_features[rand]
sample_label = test_labels[rand]
print(sample_label)

Sella


In [44]:
model = Impf_DNAMLP(len(impf), 2)
MODEL_FILE = os.path.join('../models/binary_classification_with_impf/mlp_best_states/', alg, group, f'{fold_model}.pth')
if device == 'cuda':
    model.load_state_dict(torch.load(MODEL_FILE))
else:
    model.load_state_dict(torch.load(MODEL_FILE, map_location='cpu'))
model.to(device)
probs = softmax(model(torch.Tensor(sample_feature).to(device)), dim = 0)
print(probs)

tensor([0.2077, 0.7923], grad_fn=<SoftmaxBackward0>)
